In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (10,6)
pd.set_option('display.max_columns', 100)

In [2]:
df = pd.read_csv("dummy_scout_car.csv")

In [4]:
df.sample(10)

,Unnamed: 0,price,age,km,prev_owner,hp,Consumption_comb,Nr. of Doors,Nr. of Seats,Displacement,Cylinders,Weight,Gears,cc_ Air suspension,cc_ Armrest,cc_ Automatic climate control,cc_ Auxiliary heating,cc_ Cruise control,cc_ Electric Starter,cc_ Electric tailgate,cc_ Electrical side mirrors,cc_ Electrically adjustable seats,cc_ Electrically heated windshield,cc_ Heads-up display,cc_ Heated steering wheel,cc_ Hill Holder,cc_ Keyless central door lock,cc_ Leather seats,cc_ Leather steering wheel,cc_ Light sensor,cc_ Lumbar support,cc_ Massage seats,cc_ Multi-function steering wheel,cc_ Navigation system,cc_ Panorama roof,cc_ Park Distance Control,cc_ Parking assist system camera,cc_ Parking assist system self-steering,cc_ Parking assist system sensors front,cc_ Parking assist system sensors rear,cc_ Power windows,cc_ Rain sensor,cc_ Seat heating,cc_ Seat ventilation,cc_ Split rear seats,cc_ Start-stop system,cc_ Sunroof,cc_ Tinted windows,cc_ Wind deflector,cc_ Windshield,...,body_type_Convertible,body_type_Coupe,body_type_Off-Road,body_type_Other,body_type_Sedans,body_type_Station wagon,body_type_Transporter,body_type_Van,Fuel_Diesel,Fuel_LPG,Fuel_Other,Type_Employee's car,Type_New,Type_Pre-registered,Type_Used,Body Color_Black,Body Color_Blue,Body Color_Bronze,Body Color_Brown,Body Color_Gold,Body Color_Green,Body Color_Grey,Body Color_Orange,Body Color_Red,Body Color_Silver,Body Color_Violet,Body Color_White,Body Color_Yellow,Gearing Type_Manual,Gearing Type_Semi-automatic,Upholstery_type_Full leather,Upholstery_type_Other,Upholstery_type_Part leather,Upholstery_type_Velour,Upholstery_type_alcantara,Upholstery_color_Black,Upholstery_color_Blue,Upholstery_color_Brown,Upholstery_color_Cloth,Upholstery_color_Full leather,Upholstery_color_Grey,Upholstery_color_Orange,Upholstery_color_Other,Upholstery_color_Red,Upholstery_color_White,Upholstery_color_Yellow,vat_VAT deductible,vat_VAT undeductible,Drive chain_front,Drive chain_rear
5414,5414,12980,2.0,79727.0,1.0,100.0,4.5,5.0,5.0,1598.0,4.0,1472.000000,6.0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,1,1,0,1,1,0,1,0,0,1,1,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0
3374,3374,21900,2.0,5000.0,1.0,110.0,5.0,4.0,5.0,1498.0,3.0,1401.277551,7.0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,1,1,1,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0
10724,10724,18390,2.0,14400.0,1.0,103.0,5.9,5.0,5.0,1490.0,4.0,1440.000000,6.0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
11224,11224,20890,1.0,16643.0,1.0,125.0,5.6,5.0,5.0,1956.0,4.0,1629.717452,6.0,0,1,1,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,1,0,1,0,0,1,1,1,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0
5941,5941,10791,2.0,23128.0,1.0,77.0,4.0,5.0,5.0,1399.0,3.0,1263.000000,5.0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0
718,718,18000,2.0,30000.0,1.0,66.0,3.8,5.0,4.0,1422.0,3.0,1220.000000,7.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
12561,12561,22000,2.0,20000.0,1.0,162.0,5.9,5.0,5.0,1618.0,4.0,1279.000000,6.0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,1,1,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
2146,2146,19490,0.0,1585.0,1.0,85.0,4.9,4.0,4.0,1395.0,3.0,1180.000000,6.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,1,1,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0
1

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14376 entries, 0 to 14375
Columns: 226 entries, Unnamed: 0 to Drive chain_rear
dtypes: float64(11), int64(215)
memory usage: 24.8 MB


In [8]:
df.isnull().sum().sum()

0

In [10]:
df.shape

(14376, 226)

In [11]:
X = df.drop("price", axis =1)
y = df["price"]

###  <span style='color:green'> Train|Test Split</span> 

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

## <span style='color:blue'> Ridge Regression</span> 

In [15]:
from sklearn.linear_model import Ridge

In [16]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def train_val(y_train, y_train_pred, y_test, y_pred, name):
    
    scores = {name+"_train": {"R2" : r2_score(y_train, y_train_pred),
    "mae" : mean_absolute_error(y_train, y_train_pred),
    "mse" : mean_squared_error(y_train, y_train_pred),                          
    "rmse" : np.sqrt(mean_squared_error(y_train, y_train_pred))},
    
    name+"_test": {"R2" : r2_score(y_test, y_pred),
    "mae" : mean_absolute_error(y_test, y_pred),
    "mse" : mean_squared_error(y_test, y_pred),
    "rmse" : np.sqrt(mean_squared_error(y_test, y_pred))}}
    
    return pd.DataFrame(scores)

In [17]:
ridge_model = Ridge(alpha=1, random_state=42)

In [19]:
ridge_model.fit(X_train, y_train)

Ridge(alpha=1, random_state=42)

In [20]:
y_pred = ridge_model.predict(X_test)
y_train_pred = ridge_model.predict(X_train)

In [21]:
rs = train_val(y_train, y_train_pred, y_test, y_pred, "ridge")
rs

,ridge_train,ridge_test
R2,8.996750e-01,8.928014e-01
mae,1.656069e+03,1.647098e+03
mse,5.638608e+06,5.666168e+06
rmse,2.374575e+03,2.380371e+03


### Our numbers in this model are wrong so we will try the model again after scaling the  data again

###  <span style='color:green'> Scaling the Data</span> 

In [23]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [24]:
scaler.fit(X_train)

StandardScaler()

In [25]:
X_train_scaled = scaler.transform(X_train) # Apply transform according to fit
X_test_scaled = scaler.transform(X_test)

In [26]:
ridge_model = Ridge(alpha=1, random_state=42)

In [27]:
ridge_model.fit(X_train_scaled, y_train)

Ridge(alpha=1, random_state=42)

In [28]:
y_pred = ridge_model.predict(X_test_scaled)
y_train_pred = ridge_model.predict(X_train_scaled)

In [29]:
rs_2 = train_val(y_train, y_train_pred, y_test, y_pred, "ridge")
rs_2

,ridge_train,ridge_test
R2,9.007004e-01,8.935572e-01
mae,1.641668e+03,1.643859e+03
mse,5.580972e+06,5.626219e+06
rmse,2.362408e+03,2.371965e+03


### Our numbers are still very wrong so we can conclude our data type isn't suitable to ridge-model.